# Running interactive AMBER simulations using BioSimSpace

In this notebook you'll learn how to use BioSimSpace to run various molecular simulation protocols with AMBER. 

Before we get started, let's import all of the modules that will be required.

In [ ]:
# Import BioSimSpace and rename it for convenience.
import BioSimSpace as BSS

# Import the handy glob function.
from glob import glob

## Creating a molecular system

First of all we need to load a molecular system. Some example input files are included in the `amber` directory. Let's load one of these:

In [ ]:
# Glob the input files.
files = glob("amber/ala/*")

# Create a Sire molecular system.
system = BSS.readMolecules(files)

We have now created a molecular system. The system consists of an alanine dipeptide molecule in a box of water. To show the number of molecules in the system, run:

In [ ]:
system.nMolecules()

## Defining a simulation protocol

BioSimSpace provides functionality for defining various simulation protocols. In this notebook we will construct a typical simulation workflow that uses a sequence of simple protocols, with the output of one forming the input of the next:

1. _Minimisation:_ Energy minimisation the molecular system.
2. _Equilibration:_ Equlibration of the system to a target temperature.
3. _Production:_ Regular molecular dynamics, run at fixed temperature.

When defining a protocol we are configuring the type of simulation that we wish to run, as well as any options for the particular simulation. For example, to create a default minimisation protocol:

```python
protocol = BSS.Protcol.Minimisation()
```

This defines a minimisation protocol that uses 10000 steps. For convenience, let's use 1000 steps. This can be achieved by passing the `steps` keyword argument to the constructor, i.e.:

In [ ]:
# Initialise a short minimisation protocol.
protocol = BSS.Protocol.Minimisation(steps=1000)

## Initialising a process

We now have everything that is needed to create a process object. To do so, run:

In [ ]:
process = BSS.Process.Amber(system, protocol, name="minimise")

### Setting the executable

On creation, BioSimSpace searches your `PATH` for an appropriate executable for running the process. The executable that is chosen may be dependent on the available hardware and type of workfow.

To see the executable that was chosen, run:

In [ ]:
process.exe()

To override the executable you can pass the `exe` keyword argument to the process constructor, e.g:

```python
process = BSS.Process.Amber(system, protocol, name="minimise", exe="/path/to/custom/exe")
```

### Setting the working directory

By default, BioSimSpace runs each process inside a unique temporary workspace. This is where all of the input and configuration files will be created, as well as any of the output when the process is run.

To view the working directory and the list of autogenerated input/configuration files, run:

In [ ]:
process.workDir()

In [ ]:
process.inputFiles()

When the process object is destroyed, the temporary working directory is deleted and all of the files will be lost. If you wish to specify a custom working directory, simply pass one using the `work_dir` keyword argument, e.g.:

```python
process = BSS.Process.Amber(system, protocol, name="minimise", work_dir="/path/to/custom/work/dir")
```

The directory will be created if it doesn't already exist (assuming write privileges on the path.)

### Configuring the process

For each protocol, BioSimSpace will initialise default configuration parameters appropriate to the process based on best practice in the field.

To see the list of configuration parameter strings, run:

In [ ]:
process.getConfig()

In some cases, it may be desirable to run a custom protocol. BioSimSpace provides several ways of acheiving this:

* By passing a configuration file as the `protocol` keyword argument when creating a process:

```python
process = BSS.Process.Amber(system, protocol="my_config.txt", name="minimise")
```

* By setting the configuration of an existing process:

```python
# Set the configuration from file.
process.setConfig("my_config.txt")

# Set the configuration using a list of configuration strings.
my_config = ["some config parameter string", "another config parameter string"]
process.setConfig(my_config)
```

* By adding to the configuration of an existing process:

```python
# Add using a configuration from file.
process.addToConfig("my_config.txt")

# Add a list of parameter strings to the configuration.
my_config = ["some config parameter string", "another config parameter string"]
process.addToConfig(my_config)
```

### Configuring command-line arguments

Where necessary, BioSimSpace will configure the command-line arguments needed to run the process.

To view the command-line argument string, run:

In [ ]:
process.getArgString()

The arguments are stored internally as an `OrderedDict` object. To view it, run:

In [ ]:
process.getArgs()

BioSimSpace provides functionality for setting and manipulating the arguments. For example, to disable the overwriting of output files:

In [ ]:
process.setArg('-O', False)
process.getArgs()

Let's see how the argument string changed:

In [ ]:
process.getArgString()

The `setArg` method can be used to add a new argument, or to overwrite the value of an existing argument. There are several other methods that allow the arguments to be modified:

* setArgs(args): Overwrite all arguments with a new dictionary.
* addArgs(args): Append additional arguments.
* insertArgs(arg, value, index): Insert an argument at a specific index.
* deleteArg(arg): Delete an argument from the dictionary.
* clearArgs(): Clear all of the arguments.

If you ever get in trouble, it's easy to reset the arguments to their default values:

In [ ]:
process.resetArgs()
process.getArgs()

## Running the minimisation process

Having configured the process to your liking, it's time to run a simulation.

To start the process, run:

In [ ]:
process.start()

BioSimSpace has now launched a AMBER minimisation process in the background.

To see if the process is still running:

In [ ]:
process.isRunning()

When the process has finished running we can see how many minutes it took to run.

In [ ]:
process.runTime()

We can also query the final energy of the molecular system (in kcal/mol):

In [ ]:
process.getTotalEnergy()

Congratulations, you've just run your first simulation using BioSimSpace!

## Equilibration

Suppose we want to use our minimised molecular system as the input for a new equilibration protocol. First let's grab the most recent molecular system from the process. This will be the final configuration if the process has finished, or the most recent checkpoint.

In [ ]:
minimised = process.getSystem()

Perhaps we would like to save this configuration to file for future use. The following command will write the configuration to disc, using the same file format as the files that were used to generate the original system. The output shows the name of the files that were written.

In [ ]:
 BSS.saveMolecules("minimised", minimised, system.fileFormat())

Next, we'll create a short equilibration protocol. Let's heat the system from 0 to 300K over a period of 0.05 nanoseconds, restraining the positions of backbone atoms in the alanine dipeptide.

In [ ]:
protocol = BSS.Protocol.Equilibration(runtime=0.05, temperature_start=0, temperature_end=300, restrain_backbone=True)

We can now create a new process object using the minimised system and the equilibration protocol as inputs.

In [ ]:
process = BSS.Process.Amber(minimised, protocol, name="equilibrate")

Let's check the configuration parameters for the process.

In [ ]:
process.getConfig()

Note that the initial temperature isn't exactly zero, rather a small positive value of 0.01. This is because certain molecular dynamics engines, e.g. [NAMD](http://www.ks.uiuc.edu/Research/namd) can't handle zero temperatures. If you explicity need the temperature to be zero, simply overwrite the configuration options.

If everything looks okay, let's start the process:

In [ ]:
process.start()

### Querying simulation output

We can monitor the time, temperature, and energy as the process runs. If you run this multiple times you'll see the temperature slowly increasing.

In [ ]:
print("%.2f ps, %.2f K, %.2f kcal/mol" % (1000*process.getTime(), process.getTemperature(), process.getTotalEnergy()))

It's possible to query many other thermodynamic records. What's available depends on type of protocol and the program that is used to run the protocol. To get more information, run:

In [ ]:
help(process)

### Plotting time series data

As well as querying the most recent records we can also get a time series of results by passing the `time_series` keyword argument to any of the data record getter methods, e.g.

```python
# Get a time series of pressure records.
pressure = process.getPressure(time_series=True)
```

Let's grab the same record data as above and use it to make some graphs of the data.

In [ ]:
# Import matplotlib so we can plot some data.
import matplotlib

# Short-cut for the pyplot function.
import matplotlib.pyplot as plt

# Increase the font size.
matplotlib.rcParams.update({'font.size': 20})

# Get a list of the time, temperature, and energy records.
time = process.getTime(time_series=True)
temperature = process.getTemperature(time_series=True)
energy = process.getTotalEnergy(time_series=True)

# Convert time to picoseconds.
time = [1000 * x for x in time]

# Create the figure.
fig = plt.figure(figsize=(20, 6))

# Create a plot of the temperature vs time
ax1 = plt.subplot(1, 2, 1)
ax1.plot(time, temperature, '-bo')
ax1.set(xlabel="Time (ps)", ylabel="Temperature (K)")
ax1.grid()

# Create a plot of the total energy vs time.
ax2 = plt.subplot(1, 2, 2)
ax2.plot(time, energy, '-ro')
ax2.set(xlabel="Time (ps)", ylabel="Total Energy (kcal/mol)")
ax2.grid()

# Adjust the subplot spacing.
plt.subplots_adjust(wspace=0.3)

Fantastic, you have now run an equilibration protocol. Being able to query a process in real time is an incredibly useful tool. This could enable us to check for convergence, or spot errors in the simulation. If you ever need to kill a running process (perhaps it was configured incorrectly), run:

```python
process.kill()
```

## Production

Having equilibrated our molecular system, let's now run a production molecular dynamics run. Once again, we'll grab the final system from the previous process and use it as the input to our production simulation.

In [ ]:
# Get the equilibrated system.
equilibrated = process.getSystem()

# Save to file in case we need to use it again.
BSS.saveMolecules("equilibrated", minimised, system.fileFormat())

We will now create our protocol. Let's run for 0.05 ns and tell it to restart the simulation using the velocities from the equilibrated configuration.

In [ ]:
protocol = BSS.Protocol.Production(runtime=0.05, restart=True)

As before, we can use our system and protocol to define a new process object:

In [ ]:
process = BSS.Process.Amber(equilibrated, protocol, name="production")

### Visualising the molecular system

BioSimSpace comes with several addition tools for working inside of a notebook. One of these is the `View` class that can be used to visualise the molecular system while a process is running. To create a `View` object we must attach it to a process (or a molecular system), e.g.:

In [ ]:
view = BSS.Notebook.View(process)

Let's start our simulation:

In [ ]:
process.start()

We can now visualise the system:

In [ ]:
view.system()

To only view a specific molecule:

In [ ]:
view.molecule(0)

To view a list of molecules:

In [ ]:
view.molecules([0, 5, 10])

If a particular view was of interest it can be reloaded as follows:

In [ ]:
# Reload the original view.
view.reload(0)

To save a specific view as a PDB file:

In [ ]:
view.savePDB("my_view.pdb", index=0)

That's it. We hope you've enjoyed using BioSimSpace.